In [ ]:
import os
os.chdir("..")
print(os.getcwd())


In [ ]:
%load_ext autoreload
%load_ext line_profiler

In [ ]:
%matplotlib qt
%autoreload 2

In [ ]:
from retna.networks import *
from retna.training import *
from retna.data_loader import *
from retna.view import *
from retna.prune import *
from retna.hook_tools import *
from retna.simulate_data import *

In [ ]:
import torch
import glob as glob
import matplotlib.pyplot as plt

## Load Model

In [238]:
h_chans = [30,30,20,20,20,20]
model = Retna_V1(1,6, h_chans)

In [ ]:
SimLoader = SimDataset(5)

In [ ]:
x,y = next(iter(SimLoader))

In [ ]:
plt.figure()
plt.imshow(x[0,0])
plt.figure()
plt.imshow(y[0,2], cmap="gray")

In [ ]:
train_model_x( model, SimLoader, num_epochs=50 )

In [ ]:
%lprun -f print_mosaic_x print_mosaic_x(SimLoader,model,  n=5)

In [ ]:
print_comparisons_x(model,SimLoader)

In [ ]:
print_mosaic_x(SimLoader,model,  n=5)

## Load Simulated Data into H5 Format

In [ ]:
image, mask = generate_random_data(120, 120, count=500)
#image = image.transpose(0,3,1,2)[:,0:1]

In [ ]:
plt.figure()
plt.imshow(image[0,0])
plt.figure()
plt.imshow(mask[0,2])

In [ ]:
root = ""  ## Add desination folder here
filename = root + "example_data.h5"

In [ ]:
import h5py 
with h5py.File(filename, 'w') as fh:
    dset = fh.create_dataset("input", data=image , compression="lzf" )
    dset = fh.create_dataset("target", data=mask , compression="lzf" )

## Load Data from H5 Format

In [ ]:
data = H5_Handler([filename], datasets=["input","target"])
Loader = DataLoader(data)

In [ ]:
## Specify what header the target data is in. 
Loader.dataset.dataset_names[1]= "target"  
Loader.dataset.channels[1] = None

In [ ]:
## Specify what header the target data is in. 

In [ ]:
model = model.to("cpu")
Loader.dataset.device = "cpu"

In [ ]:
train_model( model, Loader, num_epochs=10 )

In [ ]:
print_comparisons(model,Loader)

In [ ]:
print_mosaic(Loader,model,m=5,n=10)

## Load Data from Tiff Format

In [ ]:
#wip

## Load MNIST EXAMPLE

In [ ]:
import torch
import torchvision.datasets as datasets 
from torchvision.transforms import transforms

In [ ]:
mnist = datasets.MNIST(root='./data', train=True, download=False, transform=transforms.ToTensor())
loader =torch.utils.data.DataLoader(dataset=mnist, batch_size=100, shuffle=True)

In [ ]:
x,y = next(iter(loader))
plt.imshow(x[15,0])

In [242]:
h_chans = [30,30,30,20,20,20]
model = Retna_V1(1,10, h_chans)

In [208]:
def onehot(y):
  truth = np.zeros((len(y),10))
  for n in range(len(truth)):
    truth[n,y[n]] = 1

  truth = torch.Tensor(truth)
  truth = truth[...,None,None]

  return truth


In [243]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [244]:
for e in range(1000):

  optimizer.zero_grad()
  x,y = next(iter(loader))
  pred = model(x)
  pred = pred.mean(axis=(2,3), keepdim=True)
  truth = onehot(y)
  loss = dice_loss(pred,truth).mean()
  loss.backward()
  optimizer.step()

  print(loss.detach())

tensor(0.6126)
tensor(0.5328)
tensor(0.4856)
tensor(0.3757)
tensor(0.3175)
tensor(0.2085)
tensor(0.1895)
tensor(0.1914)
tensor(0.1932)
tensor(0.1912)
tensor(0.1961)
tensor(0.1987)
tensor(0.1898)
tensor(0.1900)
tensor(0.1862)
tensor(0.2063)
tensor(0.1920)
tensor(0.1966)
tensor(0.1959)
tensor(0.1936)
tensor(0.1942)
tensor(0.1849)
tensor(0.1969)
tensor(0.1948)
tensor(0.1959)
tensor(0.1910)
tensor(0.1832)
tensor(0.1901)
tensor(0.1883)
tensor(0.1790)
tensor(0.1923)
tensor(0.1905)
tensor(0.1887)
tensor(0.1900)
tensor(0.1807)
tensor(0.1928)
tensor(0.1848)
tensor(0.1876)
tensor(0.1876)
tensor(0.1859)
tensor(0.1809)
tensor(0.1886)
tensor(0.1788)
tensor(0.1883)
tensor(0.1849)
tensor(0.1913)
tensor(0.1810)
tensor(0.1874)
tensor(0.1856)
tensor(0.1906)
tensor(0.1820)
tensor(0.1731)
tensor(0.1798)
tensor(0.1919)
tensor(0.1936)
tensor(0.1794)
tensor(0.1757)
tensor(0.1844)
tensor(0.1862)
tensor(0.1895)
tensor(0.1857)
tensor(0.1894)
tensor(0.1762)
tensor(0.1834)
tensor(0.1739)
tensor(0.1833)
tensor(0.1

KeyboardInterrupt: 

In [225]:
pred = model(x)
pred = pred.mean(axis=(2,3), keepdim=True)

In [247]:
  x,y = next(iter(loader))
  pred = model(x)
  pred = pred.mean(axis=(2,3), keepdim=True)
  truth = onehot(y)
  print(np.round(pred[0].detach(),2))
  print(truth[0])

tensor([[[0.0000]],

        [[0.0000]],

        [[-0.0000]],

        [[-0.0000]],

        [[0.0000]],

        [[-0.0000]],

        [[0.0000]],

        [[-0.0000]],

        [[-0.0000]],

        [[0.8500]]])
tensor([[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[1.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]])


In [248]:
plt.imshow(x[0,0])

In [182]:
np.argmax(truth,axis=1)

array([6, 1, 1, 2, 4, 9, 1, 8, 1, 1, 8, 8, 5, 7, 3, 0, 8, 1, 3, 4, 6, 2,
       3, 2, 9, 2, 8, 1, 0, 0, 1, 6, 1, 3, 7, 5, 1, 7, 0, 7, 4, 4, 6, 1,
       5, 1, 6, 3, 4, 9, 5, 9, 0, 2, 0, 6, 4, 5, 8, 0, 2, 4, 3, 3, 2, 7,
       0, 4, 4, 1, 6, 8, 7, 2, 8, 8, 3, 4, 3, 0, 8, 0, 6, 4, 8, 5, 6, 9,
       1, 3, 1, 6, 3, 0, 4, 0, 3, 4, 7, 0], dtype=int64)

In [153]:
pred = model(x[[0]])
score = pred.mean(axis=(2,3))
score.shape

torch.Size([1, 10, 28, 28])

torch.Size([1, 10])

In [155]:
plt.imshow(pred[0,2].detach().numpy())

In [151]:
plt.imshow(pred[1,1].detach().numpy())

In [ ]:
class SimNumber(Dataset):
    def __init__(self, count=10, transform=None):
        self.count = count
        #self.input_images, self.target_masks = generate_random_data(120, 120, count=count)        
        self.transform = transform

        mnist = datasets.MNIST(root='./data', train=True, download=False, transform=transforms.ToTensor())
        loader =torch.utils.data.DataLoader(dataset=mnist, batch_size=1, shuffle=True)

        self.dataset = iter(loader)


    def __len__(self):
        return len(range(self.count))
    
    def __getitem__(self, idx): 
        range(self.count)[idx]


        image = np.zeros((30,100))
        mask  = np.zeros((9,30,100))


        for n in range(3):

            nx = n*28

            im, lb = next(self.dataset)

            image[:28, nx:nx+28]    = im
            mask[lb, :28, nx:nx+28] = im 
        

        if self.transform: pass
        #image = self.transform(image)


        image = torch.tensor(image).float()
        mask  = torch.tensor(mask).float()
        
        return [image, mask]

In [ ]:
iter_mnist = iter(loader)
x = next(iter_mnist)

In [ ]:
x,y = next(iter(SimNumber()))

In [ ]:
import numpy as np


In [ ]:
plt.imshow(x)

In [ ]:
plt.imshow(y[3])